In [225]:
import pandas as pd
import numpy as np
import altair as alt
import datetime as dt


### Data preprocessing

In [226]:
df_marvel = pd.read_csv("./data/marvel-wikia-data.csv")
df_dc = pd.read_csv("./data/dc-wikia-data.csv")


In [227]:
columns = [
    "page_id",
    "name",
    "url",
    "identity",
    "align",
    "eye",
    "hair",
    "sex",
    "gsm",
    "alive",
    "appearances",
    "first_appearance",
    "year",
]


In [228]:
df_marvel.set_axis(columns, axis=1, inplace=True)
df_marvel.drop(["first_appearance"], axis=1, inplace=True)
df_marvel.head()


,page_id,name,url,identity,align,eye,hair,sex,gsm,alive,appearances,year
0,1678,Spider-Man (Peter Parker),\/Spider-Man_(Peter_Parker),Secret Identity,Good Characters,Hazel Eyes,Brown Hair,Male Characters,NaN,Living Characters,4043.0,1962.0
1,7139,Captain America (Steven Rogers),\/Captain_America_(Steven_Rogers),Public Identity,Good Characters,Blue Eyes,White Hair,Male Characters,NaN,Living Characters,3360.0,1941.0
2,64786,"Wolverine (James \""Logan\"" Howlett)",\/Wolverine_(James_%22Logan%22_Howlett),Public Identity,Neutral Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3061.0,1974.0
3,1868,"Iron Man (Anthony \""Tony\"" Stark)",\/Iron_Man_(Anthony_%22Tony%22_Stark),Public Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2961.0,1963.0
4,2460,Thor (Thor Odinson),\/Thor_(Thor_Odinson),No Dual Identity,Good Characters,Blue Eyes,Blond Hair,Male Characters,NaN,Living Characters,2258.0,1950.0


In [229]:
df_dc.set_axis(columns, axis=1, inplace=True)
df_dc.drop(["first_appearance"], axis=1, inplace=True)
df_dc.head()


,page_id,name,url,identity,align,eye,hair,sex,gsm,alive,appearances,year
0,1422,Batman (Bruce Wayne),\/wiki\/Batman_(Bruce_Wayne),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,3093.0,1939.0
1,23387,Superman (Clark Kent),\/wiki\/Superman_(Clark_Kent),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,2496.0,1986.0
2,1458,Green Lantern (Hal Jordan),\/wiki\/Green_Lantern_(Hal_Jordan),Secret Identity,Good Characters,Brown Eyes,Brown Hair,Male Characters,NaN,Living Characters,1565.0,1959.0
3,1659,James Gordon (New Earth),\/wiki\/James_Gordon_(New_Earth),Public Identity,Good Characters,Brown Eyes,White Hair,Male Characters,NaN,Living Characters,1316.0,1987.0
4,1576,Richard Grayson (New Earth),\/wiki\/Richard_Grayson_(New_Earth),Secret Identity,Good Characters,Blue Eyes,Black Hair,Male Characters,NaN,Living Characters,1237.0,1940.0


In [230]:
# Let's merge two main studios to one dataframe, but leave column "studio" to figure out which character is which studio

df_marvel["studio"] = "marvel"
df_dc["studio"] = "dc"
print(df_marvel.shape)
df = df_marvel.append(df_dc, ignore_index=True)
print(df.shape)
df = df.rename(columns={"year": "first_appearance_year"})
df.drop(["page_id", "url", "alive", "identity"], axis=1, inplace=True)


(16376, 13)
(23272, 13)


In [231]:
# Let's refactor value to have simpler data without repeatings
df["name"] = df["name"].str.replace(r'\\"', "")
df["align"] = df["align"].str.split(" ").str[0]
df["eye"] = df["eye"].str.split(" ").str[0]
df["hair"] = df["hair"].str.split(" ").str[0]
df["sex"] = df["sex"].str.split(" ").str[0]
df["gsm"] = df["gsm"].str.split(" ").str[0]
df["appearances"] = df["appearances"].astype("Int64")
df["first_appearance_year"] = df["first_appearance_year"].astype("Int64")

df = df.sort_values(by=["appearances"], ascending=False)
df.head(50)


,name,align,eye,hair,sex,gsm,appearances,first_appearance_year,studio
0,Spider-Man (Peter Parker),Good,Hazel,Brown,Male,NaN,4043,1962,marvel
1,Captain America (Steven Rogers),Good,Blue,White,Male,NaN,3360,1941,marvel
16376,Batman (Bruce Wayne),Good,Blue,Black,Male,NaN,3093,1939,dc
2,Wolverine (James Logan Howlett),Neutral,Blue,Black,Male,NaN,3061,1974,marvel
3,Iron Man (Anthony Tony Stark),Good,Blue,Black,Male,NaN,2961,1963,marvel
16377,Superman (Clark Kent),Good,Blue,Black,Male,NaN,2496,1986,dc
4,Thor (Thor Odinson),Good,Blue,Blond,Male,NaN,2258,1950,marvel
5,Benjamin Grimm (Earth-616),Good,Blue,No,Male,NaN,2255,1961,marvel
6,Reed Richards (Earth-616),Good,Brown,Brown,Male,NaN,2072,1961,marvel
7,Hulk (Robert Bruce Banner),Good,Brown,Brown,Male,NaN,2017,1962,marvel


In [232]:
min_year = df["first_appearance_year"].min()
max_year = df["first_appearance_year"].max()
print(f'Number of not null gsm rows: {len(df[~df["gsm"].isnull()])}')
print(f"First appearance is: {min_year} - {max_year}")
print(f"Unique values of column 'sex': {df.sex.unique()}")


Number of not null gsm rows: 154
First appearance is: 1935 - 2013
Unique values of column 'sex': ['Male' 'Female' 'Genderfluid' 'Agender' nan 'Genderless' 'Transgender']


In [233]:
def new_character_per_year(df):
    min_year = df["first_appearance_year"].min()
    max_year = df["first_appearance_year"].max()
    columns = ["count", "year", "female_percentage", "gsm_count"]
    df_new_character_per_year = pd.DataFrame(columns=columns)
    female_mask = df["sex"] == "Female"
    male_mask = df["sex"] == "Male"
    gsm_mask = ~df["gsm"].isnull()


    for y in range(min_year, max_year + 1):
        year_mask = df["first_appearance_year"] == y

        c = len(df[year_mask])
        p = len(df[year_mask & female_mask]) / len(
            df[year_mask & (female_mask | male_mask)]
        )
        g = len(df[year_mask & gsm_mask])

        df_temp = pd.DataFrame([[c, y, p, g]], columns=columns)
        df_new_character_per_year = df_new_character_per_year.append(
            df_temp, ignore_index=True
        )

    return df_new_character_per_year


In [234]:
all_new_character_per_year = new_character_per_year(df)

marvel_new_character_per_year = new_character_per_year(df[df["studio"] == "marvel"])
marvel_new_character_per_year["studio"] = "marvel"
dc_new_character_per_year = new_character_per_year(df[df["studio"] == "dc"])
dc_new_character_per_year["studio"] = "dc"

df_merged = dc_new_character_per_year.append(
    marvel_new_character_per_year, ignore_index=True
)

df_merged.head()


,count,year,female_percentage,gsm_count,studio
0,1,1935,0.000000,0,dc
1,9,1936,0.222222,0,dc
2,4,1937,0.250000,0,dc
3,10,1938,0.100000,0,dc
4,18,1939,0.277778,0,dc


### New comic Book Characters Introduced Per Year

In [235]:
alt.Chart(df_merged).mark_bar().encode(
    column=alt.Column(
        "studio",
        title="",
    ),
    x=alt.X("year", title="Year", axis=alt.Axis(format="f")),
    y=alt.Y("count", title=""),
    color=alt.Color("studio", title="Comics Studio"),
).configure(background="#f9f9f9", padding=20,).configure_view(
    strokeWidth=0
).configure_axis(
    ticks=False,
    domain=False,
    gridDash=[2, 2],
).configure_header(
    labelFontSize=0,
).properties(
    width=400,
    height=250,
    title="New comic Book Characters Introduced Per Year",
).display(
    theme="fivethirtyeight"
)


alt.Chart(...)

- *Які задачі ви вирішували?*

Показати як змінювалась кількість нових героїв в коміксах з часом

- *Які способи вирішення задач ви розглядали?*

Показати все на одному графіку лініями різних кольорів

- *Чому обрали саме цей спосіб вирішення задачі?*

Тому що якщо б це було б лініями, було б великі перегиби, і тяжко було б читати з зображення

- *Які переваги та недоліки такого способу представлення даних?*

Треба порівнювати на око два графіки, але водночас дуже класно що колір студії відповідає кольорам логотипу компаній

### Gender ratio in comic books by studio

In [236]:
# Create a selection that chooses the nearest point & selects based on x-value
nearest = alt.selection(type='single', nearest=True, on='mouseover',
                        fields=['year'], empty='none')



lines = alt.Chart(df_merged).mark_line().encode(
    x=alt.X("year", title="Year", axis=alt.Axis(format="f")),
    y=alt.Y("female_percentage", title="Percentage of Female characters", axis=alt.Axis(format=".0%")),
    color=alt.Color("studio", title="Comics Studio"),
)

selectors = alt.Chart(df_merged).mark_point().encode(
    x='year:Q',
    opacity=alt.value(0),
).add_selection(
    nearest
)

# Draw points on the line, and highlight based on selection
points = lines.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Draw text labels near the points, and highlight based on selection
text = lines.mark_text(align='left', dx=10, dy=-10).encode(
    text=alt.condition(nearest, 'female_percentage:Q', alt.value(' '), format=".0%")
)

# Draw a rule at the location of the selection
rules = alt.Chart(df_merged).mark_rule(color='silver').encode(
    x='year:Q',
    size=alt.value(2)
).transform_filter(
    nearest
)

# Put the five layers into a chart and bind the data
alt.layer(
    lines, selectors, points, rules, text
).configure(background="#f9f9f9", padding=20).configure_view(
    strokeWidth=0,
).configure_axis(
    ticks=False,
    domain=False,
    gridDash=[2, 2],
).properties(
    width=700,
    height=250,
    title="The Gender Ratio In Comic Books is getting better",
).display(
    theme="fivethirtyeight"
)




alt.LayerChart(...)

- *Які задачі ви вирішували?*

Показати як змінювався відсоток жінок в коміксах з часом

- *Які способи вирішення задач ви розглядали?*

Можна було б як в попередньому графіку, але мені здається що так репрезентативніше

- *Чому обрали саме цей спосіб вирішення задачі?*

Такий вид графіку дозволяє стоврити інтерактивну лінію, яка буде показувати точний відсоток.

- *Які переваги та недоліки такого способу представлення даних?*

Перевага в тому що можна інтерактивно глянути відсоток, недолік що ліва лейба тяжко читається, легеда  

### LGBT Characters in Comics Books

In [239]:
alt.Chart(df_merged).mark_bar().encode(
    column=alt.Column(
        "studio",
        title="",
    ),
    x=alt.X("year", title="Year", axis=alt.Axis(format="f")),
    y=alt.Y("gsm_count", title=""),
    color=alt.Color("studio", title="Comics Studio"),
).configure(background="#f9f9f9", padding=20,).configure_view(
    strokeWidth=0
).configure_axis(
    ticks=False,
    domain=False,
    gridDash=[2, 2],
).configure_header(
    labelFontSize=0,
).properties(
    width=400,
    height=250,
    title="LGBT Characters in Comics Books",
).display(
    theme="fivethirtyeight"
)


alt.Chart(...)

- *Які задачі ви вирішували?*

Показати як з часом в коміксах почали виникати LGBT герої

- *Які способи вирішення задач ви розглядали?*

Показати все на одному графіку лініями різних кольорів

- *Чому обрали саме цей спосіб вирішення задачі?*

Такий вид легко читаємий, зрозумілий, і якщо б це був графік лініями, то він би мав різки переходи

- *Які переваги та недоліки такого способу представлення даних?*

Мінус в тому що можна було зробити інтерактивно, щоб бачити в року відповідає бар, а плюс в тому, що простий, мінімалістичний.

In [240]:
df.head()

,name,align,eye,hair,sex,gsm,appearances,first_appearance_year,studio
0,Spider-Man (Peter Parker),Good,Hazel,Brown,Male,NaN,4043,1962,marvel
1,Captain America (Steven Rogers),Good,Blue,White,Male,NaN,3360,1941,marvel
16376,Batman (Bruce Wayne),Good,Blue,Black,Male,NaN,3093,1939,dc
2,Wolverine (James Logan Howlett),Neutral,Blue,Black,Male,NaN,3061,1974,marvel
3,Iron Man (Anthony Tony Stark),Good,Blue,Black,Male,NaN,2961,1963,marvel


In [266]:
def good_bad_character(df):
    studios = ["marvel", "dc"]
    columns = ["percentage", "align", "sex", "studio"]
    aligns = ["Good", "Neutral", "Bad"]

    df_good_bad_character = pd.DataFrame(columns=columns)
    female_mask = df["sex"] == "Female"
    male_mask = df["sex"] == "Male"
    gsm_mask = ~df["gsm"].isnull()
    align_not_null = ~df["align"].isnull()

    for s in studios:
        studio_mask = (df["studio"] == s) & align_not_null
        female_studio_count = len(df[female_mask & studio_mask])
        male_studio_count = len(df[male_mask & studio_mask])
        gsm_studio_count = len(df[gsm_mask & studio_mask])
        for a in aligns:
            align_and_studio_mask = (df["align"] == a) & (df["studio"] == s)
            f_p = len(df[female_mask & align_and_studio_mask]) / female_studio_count
            m_p = len(df[male_mask & align_and_studio_mask]) / male_studio_count
            gsm_p = len(df[gsm_mask & align_and_studio_mask]) / gsm_studio_count

            df_temp_f = pd.DataFrame([[f_p, a, "Female", s]], columns=columns)
            df_temp_m = pd.DataFrame([[m_p, a, "Male", s]], columns=columns)
            df_temp_gsm = pd.DataFrame([[gsm_p, a, "LGBT", s]], columns=columns)

            for df_temp in [df_temp_f, df_temp_m, df_temp_gsm]:
                df_good_bad_character = df_good_bad_character.append(
                    df_temp, ignore_index=True
                )
    return df_good_bad_character


In [267]:
df_good_bad_character = good_bad_character(df)
df_good_bad_character.head(20)

,percentage,align,sex,studio
0,0.487472,Good,Female,marvel
1,0.304392,Good,Male,marvel
2,0.675000,Good,LGBT,marvel
3,0.202981,Neutral,Female,marvel
4,0.147783,Neutral,Male,marvel
5,0.212500,Neutral,LGBT,marvel
6,0.309546,Bad,Female,marvel
7,0.547824,Bad,Male,marvel
8,0.112500,Bad,LGBT,marvel
9,0.545507,Good,Female,dc


In [312]:
align_order = ["Good", "Neutral", "Bad"]
bars = (
    alt.Chart(df_good_bad_character)
    .mark_bar()
    .encode(
        column=alt.Column("sex", title=""),
        x=alt.X("percentage:Q", stack="zero", title=""),
        y=alt.Y("studio:N", title=""),
        color=alt.Color(
            "align",
            sort=align_order,
            scale=alt.Scale(domain=align_order, range=["#76ac43","#f7b901","#ff2700"]),
            title="Align"
        ),
        order="order:Q",
    )
)

bars.configure(background="#f9f9f9", padding=20).configure_view(
    strokeWidth=0,
).properties(
    width=280,
    height=80,
    title="Character alignment by Gender/Sexuality",
).configure_header(
    labelFontSize=16,
).display(
    theme="fivethirtyeight"
)


alt.Chart(...)

- *Які задачі ви вирішували?*

Показати і дізнатись певну кореляцію між gender/ sexuality та те який є герой (добрий, нейтральний, злий), до речі досить цікаві вийшли результати

- *Які способи вирішення задач ви розглядали?*

pie chart 

- *Чому обрали саме цей спосіб вирішення задачі?*

я такий chart використовував в домашній робооті і він мені сподобався ще звідти

- *Які переваги та недоліки такого способу представлення даних?*

простий, мінімалістичний, кольори інтуїтивно підходять за сприйнятям. (green=good, neutral = yellow, bad = red)
З недоліків те що все горизонтально, але в мене не вийшло зробити графіки горизонтальними. Можна було б додати відсотки на кожну з частинок, але на facet графіку таке зробити не можна.